<a href="https://colab.research.google.com/github/Harshaarigela/FMML_PROJECTS_AND_ASSIGNMENTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [3]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-3-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [4]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [6]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [7]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [12]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [14]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [15]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

answer ;Averaging validation accuracy across multiple splits of your dataset can indeed provide more consistent and robust results when evaluating the performance of a machine learning model. This technique is commonly known as cross-validation, and it helps address some of the variability and bias that can arise when assessing a model's performance on a single train-test split. Here's why it can lead to more consistent results:

Reduced Variance: When you split your data into multiple subsets (folds) and evaluate your model on each of them, you get multiple estimates of your model's performance. Averaging these estimates helps reduce the variance in your evaluation. This can be particularly important when dealing with small or noisy datasets, where a single split can lead to widely varying results.

Mitigating Bias: In a single train-test split, there's a chance that the split itself may introduce bias into the evaluation. Cross-validation helps mitigate this bias by assessing the model's performance across multiple different data partitions, making it more representative of the true performance.

Better Generalization: Averaging the results from multiple splits provides a more accurate estimate of how well your model is likely to perform on unseen data. This is particularly valuable for assessing a model's generalization capabilities.

Model Selection: Cross-validation is often used for hyperparameter tuning and model selection. By comparing the performance of different models or parameter settings across multiple splits, you can make more informed choices.

There are various types of cross-validation techniques, such as k-fold cross-validation, stratified k-fold cross-validation, leave-one-out cross-validation, and more. The choice of which type to use depends on your specific dataset and problem. For example, k-fold cross-validation splits the data into 'k' equally sized folds, while stratified k-fold ensures that each fold has a similar class distribution to the original dataset, which is particularly useful for imbalanced datasets.

In summary, averaging validation accuracy across multiple splits, as done in cross-validation, is a good practice to obtain more reliable and consistent estimates of your model's performance. It can help you make better decisions in terms of model selection, hyperparameter tuning, and assessing the generalization capability of your machine learning model.
2. Does it give more accurate estimate of test accuracy?

answer ; ross-validation, such as k-fold cross-validation, provides a more accurate estimate of test accuracy compared to a single train-test split. Here's why:

Reduced Variance: Cross-validation helps reduce the variance in the performance estimate. In a single train-test split, the specific data points included in the test set can have a significant impact on the observed performance. Cross-validation averages the results of multiple test sets, which helps stabilize the estimate by reducing the impact of any random variation in data partitioning.

Mitigating Bias: A single train-test split can introduce bias if the data is not representative or is distributed unevenly between classes. Cross-validation helps mitigate this bias by using multiple partitions of the data, making the evaluation more robust and representative.

Better Generalization: By using cross-validation, you obtain a more accurate estimate of how well your model is likely to perform on unseen data, which is crucial for assessing its generalization capabilities.

Model Selection and Hyperparameter Tuning: Cross-validation is often used for model selection and hyperparameter tuning. It allows you to compare the performance of different models or parameter settings across multiple data splits, helping you make more informed decisions.

Efficient Use of Data: Cross-validation utilizes your data more efficiently. In a single train-test split, a portion of your data is reserved for testing, while cross-validation ensures that every data point is used for both training and testing, resulting in a more effective use of your dataset.

While cross-validation provides a more accurate estimate of test accuracy, it's important to keep in mind that the estimate is still based on the same dataset. For a truly independent measure of performance, you'll need to evaluate your model on a completely new and unseen dataset, which is often referred to as a "holdout test set."

In summary, cross-validation is a valuable technique for obtaining more accurate and reliable estimates of a model's test accuracy, reducing variance and bias in the evaluation, and supporting model selection and hyperparameter tuning.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

 answerr; The number of iterations, particularly in the context of machine learning, typically refers to the number of training iterations or epochs when training a model. The effect of the number of iterations on the estimate of a model's performance depends on the specific machine learning algorithm and task. Here are some key considerations:

Underfitting and Overfitting: The number of iterations can have a significant impact on a model's performance. If you have too few iterations, your model may underfit the data, failing to capture the underlying patterns. Conversely, if you have too many iterations, your model may overfit the data, learning noise or specific details in the training data that do not generalize well to new, unseen data.

Validation Performance: It's common to monitor the model's performance on a validation set during training. Initially, as the number of iterations increases, the model's validation performance may improve, indicating that it is learning the underlying patterns. However, at a certain point, the model's performance on the validation set may start to degrade due to overfitting.

Early Stopping: To address the risk of overfitting, practitioners often use techniques like early stopping, where the training is halted when the model's performance on the validation set begins to deteriorate. This allows the model to avoid excessive iterations that could lead to overfitting.

Learning Rate and Convergence: The choice of learning rate and convergence criteria is also crucial. In some cases, a lower learning rate may require more iterations for convergence. The convergence criteria (e.g., a small change in loss) may determine when to stop training.

Complexity of the Model: More complex models, such as deep neural networks, may require a larger number of iterations to reach their optimal performance. However, they are also more prone to overfitting, so finding the right balance is essential.

Amount of Data: The size of your dataset can influence the number of iterations required. Smaller datasets may require fewer iterations, while larger datasets may require more iterations for the model to converge.

Regularization: Regularization techniques, such as L1 or L2 regularization, can affect the number of iterations required for convergence and influence the model's generalization.

In summary, there is no universal rule that more iterations always lead to a better estimate of a model's performance. The optimal number of iterations is task-dependent and must be determined through experimentation and validation. It's essential to strike a balance between training long enough to learn the underlying patterns and avoiding overfitting. Monitoring validation performance and using techniques like early stopping can help you find the right number of iterations for your specific machi
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
answer ; Increasing the number of iterations during training is not a guaranteed solution to dealing with a very small training dataset or validation dataset. While more iterations may help a model learn more from the limited data available, it can also lead to overfitting and other challenges. Here are some considerations:

Overfitting: With a very small training dataset, increasing the number of iterations can lead to overfitting. The model may start to memorize the training data rather than learning generalizable patterns. As a result, its performance on the validation dataset may degrade.

Noise Amplification: In a small dataset, there may be noise or outliers that the model can pick up. More iterations can amplify the impact of noise, making the model less robust to variations in the data.

Validation Set Size: The size of the validation dataset also matters. If the validation dataset is very small, it may not provide a reliable estimate of the model's generalization performance, regardless of the number of training iterations.

Early Stopping: To mitigate overfitting, it's important to monitor the model's performance on a validation dataset during training. You may need to use early stopping to halt training when the model's performance starts to deteriorate on the validation set, even if you haven't reached a predefined maximum number of iterations.

Regularization: Consider using regularization techniques like L1 or L2 regularization, dropout, or weight decay to help prevent overfitting when dealing with small datasets. These techniques can be more effective than increasing iterations.

Data Augmentation: If you have a very small training dataset, data augmentation techniques can be beneficial. Data augmentation involves creating new training examples by applying transformations, rotations, or other modifications to the existing data, effectively increasing the size of your training set.

Transfer Learning: In some cases, you can leverage pre-trained models on larger datasets and fine-tune them on your small dataset. Transfer learning can be a powerful approach when you have limited training data.

In conclusion, while increasing the number of iterations may help a model learn more from a small dataset, it should be done cautiously to avoid overfitting. It's crucial to monitor the model's performance on a separate validation dataset and use techniques like early stopping and regularization to ensure the model generalizes well. In cases of extremely small datasets, exploring methods like data augmentation and transfer learning may be more effective than relying solely on increasing iterations.
